In [25]:
# %conda install tensorflow
# %conda install -c conda-forge lightfm
# %conda install -c conda-forge tensorflow
# !pip install tensorflow tensorboardX # Это помогло

In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k  

from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

In [3]:
df = pd.read_csv('data/Gooddreadbooks/ratings.csv') # Поставленные оценки

In [16]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
print(train.shape)

(785404, 3)


In [5]:
# Предварительно вычислим количество уникальных пользователей и книг:
n_books = df["book_id"].nunique()
print(n_books)
n_users = df["user_id"].nunique()
print(n_users)

10000
53424


In [17]:
# В первую очередь нам необходимо создать эмбеддинги для книг и пользователей. Создаём эмбеддинги для книг:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

Сначала мы задаём размерность входного слоя. После этого определяем размер эмбеддинга — в данном случае снижаем размерность до 5. Далее мы разворачиваем результат в массив с одним измерением с помощью слоя Flatten().

Делаем то же самое для пользователей:

In [18]:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

In [19]:
# Теперь, когда мы создали представления как для книг, так и для пользователей, нам необходимо соединить их:
conc = Concatenate()([book_vec, user_vec])

Далее начинаем «собирать» нашу нейронную сеть из слоёв. Dense обозначает полносвязный слой. 

Также мы обозначаем для него количество нейронов и данные, которые идут на вход.

In [20]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

In [21]:
# Собираем модель — передаём входные данные для книг и пользователей, а также архитектуру нейронной сети:
model2 = Model([user_input, book_input], out)

Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. В данном случае будем использовать метод adam и хорошо известную вам среднеквадратичную ошибку:

In [22]:
model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

In [23]:
# Теперь будем обучать нашу модель:
history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

Epoch 1/5
24544/24544 [==============================] - 100s 4ms/step - loss: 0.7943
Epoch 2/5
24544/24544 [==============================] - 102s 4ms/step - loss: 0.6875
Epoch 3/5
24544/24544 [==============================] - 103s 4ms/step - loss: 0.6545
Epoch 4/5
24544/24544 [==============================] - 98s 4ms/step - loss: 0.6286
Epoch 5/5
24544/24544 [==============================] - 101s 4ms/step - loss: 0.6069


In [24]:
# Теперь можно оценить качество:
model2.evaluate([test.user_id, test.book_id], test.rating)

6136/6136 [==============================] - 8s 1ms/step - loss: 0.7114


0.7113780975341797

**Примечание.** К сожалению, результаты этого алгоритма нельзя зафиксировать стандартным ramdom_state, к которому мы привыкли: применяемые методы не используют такой параметр. Поэтому мы опустим здесь сравнение результатов, однако посмотрим, как можно настроить нейронную сеть.

Обычно для улучшения качества модели каким-то образом модифицируют нейронную сеть: дополняют её, увеличивают время обучения. Добавим ещё один полносвязный слой с восемью нейронами после полносвязного слоя с 32 нейронами. Обучим нейронную сеть, реализовав десять эпох:

In [26]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/10
24544/24544 [==============================] - 104s 4ms/step - loss: 0.6196
Epoch 2/10
24544/24544 [==============================] - 107s 4ms/step - loss: 0.5739
Epoch 3/10
24544/24544 [==============================] - 108s 4ms/step - loss: 0.5537
Epoch 4/10
24544/24544 [==============================] - 105s 4ms/step - loss: 0.5361
Epoch 5/10
24544/24544 [==============================] - 104s 4ms/step - loss: 0.5218
Epoch 6/10
24544/24544 [==============================] - 100s 4ms/step - loss: 0.5100
Epoch 7/10
24544/24544 [==============================] - 106s 4ms/step - loss: 0.5002
Epoch 8/10
24544/24544 [==============================] - 96s 4ms/step - loss: 0.4919
Epoch 9/10
24544/24544 [==============================] - 93s 4ms/step - loss: 0.4845
Epoch 10/10
6136/6136 [==============================] - 7s 1ms/step - loss: 0.7778


0.7778374552726746